In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count(), number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(5000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9959324971308672, 0.9949002242615376)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9992564160875681, 0.9992986358918486)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NameSurname,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Error,...,Name,NameSurname,SurnameName,Surname,NaN,NaN,PhoneNumber,Error,NaN,Error
1,NaN,CadastreCode,Document,Tax,Plate,Address,NaN,NaN,Region,Municipality,...,NaN,NameSurname,SurnameName,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
2,ItalianVAT,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NaN,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
3,ItalianVAT,NaN,Document,Tax,Plate,Address,ItalianZIPCode,Error,NaN,NaN,...,Name,Error,NaN,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,NaN,NaN,Document,Tax,Plate,Address,ItalianZIPCode,NaN,Region,NaN,...,Name,NameSurname,SurnameName,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
5,ItalianVAT,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NaN,SurnameName,Surname,String,EMail,PhoneNumber,Date,Error,Error
6,Error,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,Name,NaN,Error,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NameSurname,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,SurnameName,...,Name,SurnameName,SurnameName,Surname,NaN,NaN,PhoneNumber,Error,NaN,Error
1,NaN,CadastreCode,Document,Tax,Plate,Address,NaN,NaN,Region,Municipality,...,NaN,SurnameName,SurnameName,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
2,ItalianVAT,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NaN,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
3,ItalianVAT,NaN,Document,Tax,Plate,Address,ItalianZIPCode,Error,NaN,NaN,...,Name,Error,NaN,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,NaN,NaN,Document,Tax,Plate,Address,ItalianZIPCode,NaN,Region,NaN,...,Name,SurnameName,SurnameName,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
5,ItalianVAT,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NaN,SurnameName,Surname,String,EMail,PhoneNumber,Date,Error,Error
6,Error,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,Name,NaN,Error,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('NaN', 'CountryCode'): 82,
         ('SurnameName', 'NameSurname'): 571,
         ('Error', 'Document'): 131,
         ('NameSurname', 'SurnameName'): 637,
         ('Error', 'NaN'): 9,
         ('SurnameName', 'Error'): 182,
         ('Error', 'BiologicalSex'): 5,
         ('Error', 'ItalianZIPCode'): 10,
         ('CountryCode', 'NaN'): 78,
         ('Error', 'NameSurname'): 138,
         ('Error', 'CountryCode'): 42,
         ('Error', 'ItalianVAT'): 26,
         ('Error', 'SurnameName'): 129,
         ('Float', 'Error'): 13,
         ('NameSurname', 'Error'): 212,
         ('PhoneNumber', 'Error'): 8,
         ('Error', 'Boolean'): 8,
         ('Error', 'ProvinceCode'): 45,
         ('Error', 'String'): 26,
         ('Error', 'Country'): 7,
         ('Error', 'ItalianFiscalCode'): 32,
         ('NaN', 'Error'): 17,
         ('PhoneNumber', 'String'): 8,
         ('Error', 'Date'): 3,
         ('Surname', 'Name'): 4,
         ('Tax', 'Error'): 10,
         ('Plate', 'Strin